In [13]:
# import các package cần thiết
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np #Thư viện đại số tuyến tính và một số xác suất (dữ liệu thô)
import imutils
import time
import cv2
import os


In [14]:
def detect_and_predict_mask(frame, faceNet, maskNet):
    # lấy các kích thước của frame và sau đó tạo ra một blob từ bản thể
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),
        (104.0, 177.0, 123.0))

    # đưa blob vào network và tạo ra face detections
    # detect face rồi căt và detect mask trên cái đã cắt
    faceNet.setInput(blob)
    detections = faceNet.forward()
    print(detections.shape)

    # khởi tạo danh sách các khuôn mặt, vị trí tương ứng của chúng và danh sách các dự đoán 
    # từ mask network
    faces = []
    locs = []
    preds = []

    # vòng lặp detections
    for i in range(0, detections.shape[2]):
            # trích xuất độ tin cậy (tức là xác suất) liên quan đến phát hiện
        confidence = detections[0, 0, i, 2]

        # lọc ra các detections yếu bằng cách đảm bảo độ tin cậy lớn hơn độ tin cậy tối thiểu
        if confidence > 0.5:
            # tính toán tọa độ (x, y) của box cho đối tượng
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # đảm bảo các box nằm trong kích thước của frame
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

            # trích xuất ROI của khuôn mặt, chuyển đổi nó từ thứ tự kênh BGR sang RGB, 
            # thay đổi kích thước thành 224x224 và xử lý trước
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)

            # thêm các faces và bounding boxes vào danh sách tương ứng của chúng
            faces.append(face)
            locs.append((startX, startY, endX, endY))

    # chỉ đưa ra dự đoán nếu ít nhất một khuôn mặt được phát hiện
    if len(faces) > 0:
        # để suy luận nhanh hơn, đưa ra dự đoán hàng loạt trên tất cả các mặt cùng một lúc thay vì 
        # dự đoán từng cái một trong vòng lặp `for` ở trên
        faces = np.array(faces, dtype="float32")
        preds = maskNet.predict(faces, batch_size=32)

    # trả về 2 bộ vị trí khuôn mặt và các vị trí tương ứng của chúng
    return (locs, preds)


In [15]:
# load our serialized face detector model
prototxtPath = r".\deploy.prototxt"
weightsPath = r".\res10_300x300_ssd_iter_140000.caffemodel"
# dùng readnet để detect face, * tìm hiểu resnet kết hợp ssd
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)


In [16]:
# load the face mask detector model 
maskNet = load_model("model.model")

In [17]:
# Khởi tạo video stream
vs = VideoStream(src=0).start()

In [18]:
# vòng lặp frames từ video stream
while True:
    # lấy khung hình từ luồng video theo chuỗi và 
    # thay đổi kích thước để có chiều rộng tối đa là 400 pixel
    frame = vs.read()
    frame = imutils.resize(frame, width=400)

    # phát hiện các khuôn mặt trong khung hình và xác định xem họ có đeo khẩu trang hay không
    (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)

    # lặp lại các vị trí khuôn mặt được phát hiện và các vị trí tương ứng của chúng
    for (box, pred) in zip(locs, preds):
        #  giải nén bounding box và dự đoán
        (startX, startY, endX, endY) = box
        (mask, withoutMask) = pred

        # xác định class label và màu cho bounding box, label
        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

        # thêm probability(xác xuất, độ chính xác) vào label
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

        # hiển thị label và bounding box ra frame
        cv2.putText(frame, label, (startX, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

    # hiển thị title của frame và frame
    cv2.imshow("Face Mask Detector", frame)
    key = cv2.waitKey(1) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break


(1, 1, 200, 7)
(1, 1, 200, 7)
1/1 [==============================] - 1s 1s/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 31ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 28ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 31ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 29ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 35ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 31ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 29ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 29ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 32ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 41ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 30ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 35ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 32ms/step
(1, 1, 200, 7)
1/1 [=============================

1/1 [==============================] - 0s 30ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 31ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 31ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 29ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 30ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 28ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 30ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 31ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 31ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 29ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 31ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 30ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 32ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 30ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 30ms/step
(1, 1, 200

1/1 [==============================] - 0s 30ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 30ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 31ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 31ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 30ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 30ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 30ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 31ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 30ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 30ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 32ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 30ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 29ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 34ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 29ms/step
(1, 1, 200

1/1 [==============================] - 0s 30ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 30ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 29ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 31ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 28ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 32ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 31ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 31ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 31ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 29ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 30ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 37ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 35ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 33ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 33ms/step
(1, 1, 200

In [19]:
# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()